# ME308 PROJECT: AIRPLANE SCHEDULING AND COST MINIMIZATION

## Problem Statement 1
Given a set of routes pq on which there exists an average daily traffic Ppq which must be carried by a fleet consisting of limited numbers of different types of aircraft, find the least cost assignment of aircraft to routes such that a minimum frequency of daily service on each route is satisfied, and such that aircraft movements balance at every station while not exceeding a daily allowable maximum imposed
by airport congestion.

### Declaring all variables

In [103]:
import numpy

#Adjustable parameters
cities = 3  # no of cities being catered

# The load factor is a metric used in the airline industry that measures the percentage of available seating capacity that has been filled with passengers
LF_pq = numpy.zeros((cities, cities))
# estimated average for daily traffic from p to q
P_pq = numpy.zeros((cities, cities))
# an appropriate direct operating cost for an aircraft a on route pq
DC_pq = numpy.zeros((cities, cities))
# Stores distance between 2 airports p and q
d_pq = numpy.zeros((cities, cities))
# stores seating capacity for a given aircraft
S = 180
# estimated average for daily traffic from p to q
P_pq = numpy.zeros((cities, cities))
# block hours for aircraft a to fly route pq
# Block time includes the time to taxi-out to the runway, the actual flight duration and the time to taxi to the arrival gate, but the published schedule for the flight doesn't break these elements apart.
Tb_pq = numpy.zeros((cities, cities))
# assumed average daily utilization for fleet a [in hours] - depends on stage length as well (the amount of time plane has flown)
U = 8
# number of aircrafts
A = 50
# a specified minimum level of service should be maintained on each route
Nmin_pq = numpy.zeros((cities, cities))
# maximum number of daily departures allowed at station p
N_maxp = numpy.zeros(cities)

### Initialization of variables

Some information before code
We take three cities for testing - Mumbai, Delhi, Lucknow <br>
Distance(Mumbai->Delhi) = 1137Km <br>
Distance(Delhi->Lucknow) = 553Km <br>
Distance(Mumbai->Lucknow) = 1378km <br>
<br>
We assume presence of two fleets in our airlines - A320 200 and B737 (say)<br>
Seating capacity of each of them is equal to 180 <br><br>
Since Delhi and Lucknow are close to each other people also prefer superfast trains as compared to flight since it takes same time. So load factor for Delhi to Lucknow might be low.<br>
But in our case we assume constant load factor = 1<br>
Average daily traffic from one city to other is taken as 1000 for Mumbai to Delhi, 400 for Delhi to Lucknow and 700 for Mumbai to Lucknow<br><br>
One major assumption is aircrafts fly at same velocity throughout the journey and velocity of aircraft is ~900km/hr <br>
Fuel cost = INR 40 per litre (assuming full seats) # is not valid when LF<1 though <br>
Fuel consumption = 4.18 L/km (Data for A320) <br>
DC = fuel consumption * distance * fuel cost <br>
<br>
Block time = distance/velocity + 1hr<br><br>
Assumed average utilisation is 8hrs
<br><br>
Minimum level of service = 1
<br>
Maximum level of service = 900 for Mumbai, 800 for Delhi and 100 for Lucknow
<br>
No of aircrafts = 50



In [104]:
import numpy as np
# The order is Mumbai, Delhi, Lucknow in each case
LF_pq = np.array(np.ones((cities, cities)))
fuel_cost = 40
fuel_consumption = 4.18
DC_pq = np.array(np.dot(d_pq, fuel_cost*fuel_consumption))
d_pq = numpy.array([
    [0, 1137, 1378],
    [1137, 0, 553],
    [1378, 553, 0]
])
S = 180
P_pq = np.array([
    [0, 1000, 700],
    [1000, 0, 400],
    [700, 400, 0]
]) # assume symmetric demand as of now
Tb_pq = d_pq/900+1 #addition of 1 is for gate closing and onboarding time
for i in range(cities):
  Tb_pq[i][i] = 0 #since no flight is between them
U = 8
A = 50
Nmin_pq = np.array(np.ones((cities, cities)))
for i in range(cities):
  Nmin_pq[i][i] = 0 #no flights to same airport
Nmax_p = np.array([900, 800, 100])

### Initialising pulp and creating decision variable

In [105]:
# pip install pulp

In [106]:
from pulp import *
# decision variable
n_pq = LpVariable.dicts("n", (range(cities), range(cities)), cat="Integer")
print(n_pq)

{0: {0: n_0_0, 1: n_0_1, 2: n_0_2}, 1: {0: n_1_0, 1: n_1_1, 2: n_1_2}, 2: {0: n_2_0, 1: n_2_1, 2: n_2_2}}


## Optimization problem
Minimize operating cost for fixed revenue $$\sum_{pq}DC_{pq}n_{pq}$$
The model is of LP minimization type with the following constraints<br>
1. Demand on all routes must be carried (Load factor is 1)
$$\sum S n_{pq} \ge P_{pq} \forall pq$$
2. Fleet availability must not be exceeded
$$\sum_{pq} Tb_{pq}n_{pq} \le UA$$
3. Aircraft movements must balance over the day at every station
$$\sum_{q}n_{pq} = \sum_{q}n_{qp}$$
4. Minimum daily frequency must be maintained on each route
$$\sum n_{pq} \ge Nmin_{pq}$$
5. Maximum daily departures at a station are limited due to airport capacity
$$\sum_{q}n_{pq} \le Nmax_{p}$$

In [107]:
model = LpProblem("Airline Cost Minimizer", LpMinimize)
model += lpSum([DC_pq[i][j] * n_pq[i][j] for i in range(cities) for j in range(cities)]); #optimization function
for i in range(cities):
  for j in range(cities):
    model += S*n_pq[i][j]>=P_pq[i][j];
    model += n_pq[i][j] >= Nmin_pq[i][j]
model += lpSum([Tb_pq[i][j] * n_pq[i][j] for i in range(cities) for j in range(cities)]) <= U*A;
for i in range(cities):
  model += lpSum(n_pq[i][j] for j in range(cities)) == lpSum(n_pq[j][i] for j in range(cities))
  model += lpSum(n_pq[i][j] for j in range(cities)) <= Nmax_p[i]
print(model);

Airline_Cost_Minimizer:
MINIMIZE
0
SUBJECT TO
_C1: 180 n_0_0 <= 0

_C2: n_0_0 >= 0

_C3: 180 n_0_1 <= 1000

_C4: n_0_1 >= 1

_C5: 180 n_0_2 <= 700

_C6: n_0_2 >= 1

_C7: 180 n_1_0 <= 1000

_C8: n_1_0 >= 1

_C9: 180 n_1_1 <= 0

_C10: n_1_1 >= 0

_C11: 180 n_1_2 <= 400

_C12: n_1_2 >= 1

_C13: 180 n_2_0 <= 700

_C14: n_2_0 >= 1

_C15: 180 n_2_1 <= 400

_C16: n_2_1 >= 1

_C17: 180 n_2_2 <= 0

_C18: n_2_2 >= 0

_C19: 2.26333333333 n_0_1 + 2.53111111111 n_0_2 + 2.26333333333 n_1_0
 + 1.61444444444 n_1_2 + 2.53111111111 n_2_0 + 1.61444444444 n_2_1 <= 400

_C20: 0 n_0_0 + n_0_1 + n_0_2 - n_1_0 - n_2_0 = 0

_C21: n_0_0 + n_0_1 + n_0_2 <= 900

_C22: - n_0_1 + n_1_0 + 0 n_1_1 + n_1_2 - n_2_1 = 0

_C23: n_1_0 + n_1_1 + n_1_2 <= 800

_C24: - n_0_2 - n_1_2 + n_2_0 + n_2_1 + 0 n_2_2 = 0

_C25: n_2_0 + n_2_1 + n_2_2 <= 100

VARIABLES
n_0_0 free Integer
n_0_1 free Integer
n_0_2 free Integer
n_1_0 free Integer
n_1_1 free Integer
n_1_2 free Integer
n_2_0 free Integer
n_2_1 free Integer
n_2_2 free Intege

### Solving the model

In [108]:
status = model.solve()

In [109]:
status

1

In [110]:
for v in model.variables():
    print(v.name, "=", v.varValue)

__dummy = None
n_0_0 = 0.0
n_0_1 = 1.0
n_0_2 = 1.0
n_1_0 = 1.0
n_1_1 = 0.0
n_1_2 = 1.0
n_2_0 = 1.0
n_2_1 = 1.0
n_2_2 = 0.0


# Problem Statement 2
Given a set of non-stop routes pq for which there exists a market share curve relating traffic carried to daily frequency n , an aircraft fleet consisting of limited numbers of different types of aircraft, find the maximum income assignment of aircraft to routes such that a minimum daily frequency of service is maintained, and such that aircraft movements balance at every station while not exceeding an allowable maximum imposed by airport congestion

 Breakeven Load Factor – the number of seats they have to sell to cover operating expenses.